# Workspace Access Review Audit

## Purpose
This notebook performs a comprehensive access review audit for workspace folders in Databricks. It identifies and tracks all notebook modifications within a configurable date range, with optional long-term retention and content snapshots.

## Execution Modes

This notebook automatically detects and optimizes for two execution modes:

* **Interactive Mode**: When run manually in the notebook UI
  * Displays results with `display()` and `print()` statements
  * Shows progress messages and sample data
  * Ideal for exploration and ad-hoc analysis

* **Job Mode**: When run as a scheduled job or workflow
  * Suppresses all `print()` and `display()` output for performance
  * Optimized for automated execution
  * No code changes required - automatically detected
  * Automatically exports to Delta tables when enabled

## What This Code Does

1. **Recursive Notebook Scanning**: Scans all notebooks within a specified folder (or all workspace folders) and its subdirectories to build a complete inventory of workspace assets.

2. **Audit Log Analysis**: Queries the `system.access.audit` table to retrieve all modification events including:
   * Notebook updates and edits
   * New notebook creation
   * Notebook deletions
   * Command executions
   * Notebook attach/detach operations

3. **Enhanced Change Tracking**: Analyzes modification patterns over time:
   * Modification activity by day of week and hour
   * Daily modification trends
   * Peak activity periods
   * User activity patterns

4. **Content Snapshots**: Captures point-in-time notebook metadata:
   * Current notebook state (size, language, last modified)
   * Modification history per notebook
   * Days active and modification frequency
   * Complete modifier list

5. **Long-Term Retention**: Exports audit data to Delta tables:
   * Historical accumulation of audit events
   * Snapshot tables for current state
   * Enables trend analysis across multiple audit runs
   * Query historical changes over time

6. **Detailed Reporting**: Produces multiple views of the audit data:
   * Per-event log with notebook path, action type, user, and timestamp
   * Summary statistics by action type
   * User activity analysis
   * Per-notebook modification history
   * Time-based modification patterns

## Configuration
Set these variables in Cell 3 to customize the audit scope:

### Basic Configuration

* **`BASE_PATH`**: Specify the folder to audit
  * Set to a specific path (e.g., `"/Shared/ProjectName"`) to audit a particular folder
  * Set to `""` (empty string) to audit all workspace folders

* **`START_DATE`**: Beginning of the audit period (format: `"YYYY-MM-DD"`)
  * Example: `"2025-01-01"`

* **`END_DATE`**: End of the audit period (format: `"YYYY-MM-DD"`, exclusive)
  * Example: `"2026-01-01"` (will query up to but not including this date)

* **`TIMEZONE`**: Timezone for displaying event timestamps (default: `"America/New_York"`)
  * Common values: `"America/New_York"` (Eastern), `"America/Chicago"` (Central), `"America/Denver"` (Mountain), `"America/Los_Angeles"` (Pacific), `"UTC"`, `"Europe/London"`, etc.
  * Audit logs are stored in UTC and automatically converted to the specified timezone

* **`MAX_DETAILED_EVENTS`**: Maximum number of events to display in detailed logs (default: `1000`)
  * Set to `9999` to retrieve **ALL events without any limit**
  * Any other value limits the results to that number (prevents memory issues with large datasets)
  * Increase if you need more detailed history, decrease for faster performance
  * **Note**: Using `9999` (no limit) may cause memory issues with very large audit datasets

### Advanced Features

* **`ENABLE_DELTA_EXPORT`**: Export audit data to Delta table (default: `False`)
  * Set to `True` to enable long-term retention
  * Accumulates historical audit data across multiple runs
  * Enables trend analysis and compliance reporting

* **`DELTA_TABLE_NAME`**: Target Delta table name (default: `"main.default.notebook_audit_history"`)
  * Format: `"catalog.schema.table"`
  * Requires CREATE TABLE permissions
  * Automatically creates snapshot table with `_snapshot` suffix

* **`ENABLE_CONTENT_SNAPSHOT`**: Capture notebook metadata (default: `True`)
  * Captures current notebook state (size, language, modified time)
  * Combines with audit history for comprehensive view
  * Provides point-in-time snapshot of workspace state

## Performance Optimizations

* **Partition Pruning**: Date range filters enable efficient partition pruning on the audit table
* **Selective Column Selection**: Queries select only needed columns to reduce data transfer
* **Configurable Limits**: MAX_DETAILED_EVENTS prevents memory issues with large result sets (unless set to 9999)
* **No Unnecessary Caching**: Optimized for serverless compute environments
* **Recursion Protection**: Max depth limit prevents infinite loops in folder scanning
* **Mode-Specific Execution**: Job mode skips all display operations for better performance
* **Delta Table Optimization**: Uses append mode for history, overwrite for snapshots

## Output
The audit report provides:
* Complete trail of who accessed and modified notebooks
* Time-based modification patterns and trends
* Point-in-time snapshots of notebook state
* Optional long-term retention in Delta tables
* Suitable for compliance, security reviews, and access governance

## Use Cases

1. **Compliance Auditing**: Track all modifications for regulatory requirements
2. **Security Reviews**: Identify unusual access patterns or unauthorized changes
3. **Team Analytics**: Understand team collaboration patterns and peak activity times
4. **Change Management**: Monitor notebook lifecycle and modification frequency
5. **Historical Analysis**: Query trends across multiple audit periods using Delta tables

## Version Control

| Version | Date | Author | Changes |
|---------|------|--------|----------|
| 1.0 | 2026-02-10 | Brandon Croom | Created comprehensive audit framework with recursive notebook scanning using Databricks SDK WorkspaceClient; Configurable BASE_PATH for specific folders or all workspace folders; Configurable date range (START_DATE, END_DATE) and timezone (TIMEZONE); Audit log queries for notebook modification events (modifyNotebook, createNotebook, deleteNotebook, runCommand, etc.); Dynamic path filtering with trailing slash handling; UTC to local timezone conversion for all timestamps; Multiple reporting views (per-event log, summary statistics, user activity, per-notebook history); Performance optimizations (removed caching for serverless compatibility, optimized queries); Report generation timestamp in configured timezone |
| 1.1 | 2026-02-11 | Brandon Croom | Added automatic job vs interactive mode detection; Conditional display/print statements for performance in job mode; Removed unnecessary cache operations; Added MAX_DETAILED_EVENTS configuration to prevent memory issues with special value 9999 to retrieve ALL events without any limit; Conditional LIMIT clause in detailed log and final report queries; Enhanced error handling (permission denied, max recursion depth protection); Optimized final report query to select only needed columns; Added performance comments throughout code; Improved WorkspaceClient initialization with error handling; Updated documentation to explain unlimited data retrieval option; Dynamic messaging based on limit setting; Enhanced change tracking with time-based analysis (modification patterns by day/hour, daily trends); Implemented content snapshot feature to capture point-in-time notebook metadata (size, language, modified time) combined with audit history; Added Delta table export for long-term retention with configurable ENABLE_DELTA_EXPORT and DELTA_TABLE_NAME; Automatic snapshot table creation with _snapshot suffix; Historical accumulation mode for audit events; Schema merge support for evolving data structures; Enhanced notebook scanning to capture metadata when snapshots enabled; Comprehensive documentation of new features and use cases |
| 1.2 | 2026-02-12 | Brandon Croom | Added serverless vs traditional cluster detection with automatic compute type identification; Implemented compute-aware optimizations (no caching on serverless, automatic memory management); Added clear logging about compute environment and execution mode; Enhanced environment information display showing job mode and compute type; Improved portability across all compute types (serverless, traditional clusters, interactive clusters); Updated documentation to reflect serverless optimization capabilities; Added comprehensive enhancement suite: Stale notebook detection with multiple thresholds (90/180/365 days) for compliance and workspace cleanup; Notebook ownership identification tracking who created each notebook for accountability; Execution failure analysis identifying notebooks with failed runCommand events for quality insights; Language distribution analysis; Folder-level statistics with notebook counts and language usage per folder; Notebook size analysis with large notebook detection (>1MB); Security and compliance analysis including after-hours modification detection (outside 8 AM-6 PM), notebook deletion tracking (high-risk events), and external user activity detection with configurable company domain; Collaboration metrics identifying notebooks with multiple editors and highly collaborative notebooks (3+ editors); Visual analytics with activity heatmaps (day/hour patterns, daily trends, 4-chart dashboard); Enhanced Excel export with 9-sheet comprehensive workbook (Executive Summary, Events, Users, Notebooks, Stale, Ownership, Failures, Languages, Security); JSON export for API integration and programmatic access; All features gracefully handle empty data and work on serverless compute; Genericized all code by removing EAP references and replacing email addresses with author name; Updated BASE_PATH to generic empty string with helpful examples; Complete documentation updates |

---

In [0]:
# Recursively scan all notebooks in workspace folders
import os
from datetime import datetime
from databricks.sdk import WorkspaceClient

# ============================================================================
# MODE DETECTION: Automatically detect job vs interactive mode
# ============================================================================
try:
    # Check if running in a job context
    dbutils.notebook.entry_point.getDbutils().notebook().getContext().currentRunId().isDefined()
    IS_JOB_MODE = True
except:
    IS_JOB_MODE = False

# ============================================================================
# COMPUTE DETECTION: Automatically detect serverless vs traditional cluster
# ============================================================================
try:
    test_df = spark.range(1)
    test_df.cache()
    test_df.count()
    test_df.unpersist()
    IS_SERVERLESS = False
except Exception as e:
    # If caching fails with serverless error, we're on serverless
    IS_SERVERLESS = 'SERVERLESS' in str(e) or 'PERSIST TABLE is not supported' in str(e)

# ============================================================================
# CONFIGURATION: Set the folder path, date range, and timezone
# ============================================================================
# Set to a specific folder path or leave blank ("") to scan all workspace folders
# Examples:
#   BASE_PATH = "/Shared/YourFolder/"        # Scan a specific shared folder
#   BASE_PATH = "/Users/your.email@company.com/"  # Scan your user folder
#   BASE_PATH = ""                           # Scan ALL workspace folders
BASE_PATH = ""

# Set the date range for audit log queries (format: 'YYYY-MM-DD')
START_DATE = "2025-01-01"
END_DATE = "2026-01-01"  # Exclusive - will query up to but not including this date

# Set the timezone for displaying event timestamps
# Common values: 'America/New_York' (Eastern), 'America/Chicago' (Central),
#                'America/Denver' (Mountain), 'America/Los_Angeles' (Pacific),
#                'UTC', 'Europe/London', etc.
TIMEZONE = "America/New_York"

# Maximum number of events to display in detailed log
# Set to 9999 to retrieve ALL events without any limit
# Any other value will limit the results to that number (prevents memory issues)
MAX_DETAILED_EVENTS = 9999

# ============================================================================
# LONG-TERM RETENTION: Export audit data to Delta table
# ============================================================================
# Enable exporting audit results to a Delta table for long-term retention
ENABLE_DELTA_EXPORT = False

# Target Delta table name (format: catalog.schema.table)
# Example: "main.audit_reports.notebook_modifications"
DELTA_TABLE_NAME = "main.default.notebook_audit_history"

# ============================================================================
# CONTENT SNAPSHOTS: Capture notebook metadata at time of audit
# ============================================================================
# Enable capturing current notebook metadata (size, language, modified time)
# This provides a point-in-time snapshot of notebook state
ENABLE_CONTENT_SNAPSHOT = True
# ============================================================================

def log(message):
    """Print only in interactive mode"""
    if not IS_JOB_MODE:
        print(message)

def list_workspace_notebooks_recursive(w, path, depth=0, max_depth=50):
    """
    Recursively list all notebooks in a given workspace path.
    Returns a list of dictionaries with notebook details.
    
    Args:
        w: WorkspaceClient instance
        path: Path to scan
        depth: Current recursion depth (for protection against infinite loops)
        max_depth: Maximum recursion depth allowed
    """
    # Protection against infinite recursion
    if depth > max_depth:
        log(f"⚠️  Max recursion depth reached at {path}")
        return []
    
    notebooks = []
    
    try:
        # Use Databricks SDK WorkspaceClient to list workspace objects
        items = w.workspace.list(path)
        
        for item in items:
            if item.object_type and item.object_type.name == "DIRECTORY":
                # Recursively scan subdirectories
                notebooks.extend(list_workspace_notebooks_recursive(w, item.path, depth + 1, max_depth))
            elif item.object_type and item.object_type.name == "NOTEBOOK":
                # Notebook object - capture enhanced metadata if enabled
                notebook_info = {
                    'path': item.path,
                    'name': os.path.basename(item.path),
                    'object_type': item.object_type.name,
                    'language': getattr(item, 'language', 'UNKNOWN')
                }
                
                # Add snapshot metadata if enabled
                if ENABLE_CONTENT_SNAPSHOT:
                    notebook_info['modified_at'] = getattr(item, 'modified_at', None)
                    notebook_info['size'] = getattr(item, 'size', None)
                    notebook_info['created_at'] = getattr(item, 'created_at', None)
                
                notebooks.append(notebook_info)
    except Exception as e:
        # If path doesn't exist or can't be accessed
        error_str = str(e)
        if "does not exist" in error_str or "RESOURCE_DOES_NOT_EXIST" in error_str:
            log(f"⚠️  Folder not found: {path}")
            log(f"   Please verify the folder exists in your workspace.")
        elif "PERMISSION_DENIED" in error_str:
            log(f"⚠️  Permission denied: {path}")
        else:
            log(f"Error accessing {path}: {e}")
        return notebooks
    
    return notebooks

# Initialize WorkspaceClient with error handling
try:
    w = WorkspaceClient()
except Exception as e:
    log(f"❌ Failed to initialize WorkspaceClient: {e}")
    raise

# Display environment information
log("="*60)
log("WORKSPACE ACCESS REVIEW AUDIT")
log("="*60)
log(f"Execution Mode: {'JOB' if IS_JOB_MODE else 'INTERACTIVE'}")
log(f"Compute Type: {'SERVERLESS' if IS_SERVERLESS else 'TRADITIONAL CLUSTER'}")

if IS_SERVERLESS:
    log("\n⚡ Serverless optimizations enabled:")
    log("  - Automatic memory management")
    log("  - No explicit caching needed")
    log("  - Optimized for fast startup")
else:
    log("\n🔧 Traditional cluster:")
    log("  - Manual memory management available")
    log("  - Persistent compute resources")

log("="*60)

# Determine scan path
if BASE_PATH == "" or BASE_PATH is None:
    scan_path = "/"
    log("Scanning ALL workspace folders (this may take a while)...")
else:
    scan_path = BASE_PATH
    log(f"Scanning workspace folder: {scan_path}")

log("="*60)

notebooks = list_workspace_notebooks_recursive(w, scan_path)

if len(notebooks) == 0:
    log(f"\n❌ No notebooks found in {scan_path}")
    log(f"\nPossible reasons:")
    log(f"  1. The folder doesn't exist in this workspace")
    log(f"  2. The folder is empty")
    log(f"  3. You don't have permission to access it")
    log(f"\n💡 To verify, check if the folder exists in the workspace browser.")
else:
    log(f"\n✅ Found {len(notebooks)} notebook(s) in {scan_path}")
    if not IS_JOB_MODE:
        log(f"\nSample notebooks:")
        for nb in notebooks[:10]:
            log(f"  - {nb['path']} ({nb.get('language', 'UNKNOWN')})")
        
        if len(notebooks) > 10:
            log(f"  ... and {len(notebooks) - 10} more")

# Store notebook count for downstream use
notebook_count = len(notebooks)

In [0]:
# Query audit logs for notebook modification events
# Uses the BASE_PATH, START_DATE, END_DATE, and TIMEZONE variables from the previous cell

# Build the path filter condition
# Remove trailing slash from BASE_PATH if present
base_path_clean = BASE_PATH.rstrip('/') if BASE_PATH else ""

if base_path_clean == "" or base_path_clean is None:
    path_filter = ""  # No path filter - scan all notebooks
    filter_description = "all workspace folders"
else:
    path_filter = f"AND request_params.path LIKE '{base_path_clean}/%'"
    filter_description = base_path_clean

log(f"Querying audit logs for: {filter_description}")
log(f"Date range: {START_DATE} to {END_DATE} (exclusive)")
log(f"Timezone: {TIMEZONE}")
log("="*60)

# Create the temp view with dynamic path filter and date range
# NOTE: service_name is 'notebook' (singular), not 'notebooks'
# Action names: modifyNotebook, createNotebook, deleteNotebook, etc.
# This query focuses on MODIFICATION events only (not read/access events like openNotebook)
# PERFORMANCE OPTIMIZATIONS:
# - Removed ORDER BY from view definition - ordering happens in individual queries
# - event_date filter enables partition pruning on the audit table
# - service_name and action_name filters reduce data scanned
# TIMEZONE: Convert event_time from UTC to configured timezone
query = f"""
CREATE OR REPLACE TEMP VIEW audit_events AS
SELECT 
  from_utc_timestamp(event_time, '{TIMEZONE}') as event_time,
  event_date,
  action_name,
  user_identity.email as user_email,
  request_params.path as notebook_path,
  request_params.notebook_id as notebook_id,
  service_name,
  request_id,
  response.status_code
FROM system.access.audit
WHERE 
  event_date >= '{START_DATE}' 
  AND event_date < '{END_DATE}'
  AND service_name = 'notebook'
  AND action_name IN (
    'modifyNotebook',
    'createNotebook', 
    'deleteNotebook',
    'runCommand',
    'submitCommand',
    'attachNotebook',
    'detachNotebook',
    'renameNotebook',
    'moveNotebook',
    'cloneNotebook',
    'importNotebook'
  )
  {path_filter}
"""

spark.sql(query)

# PERFORMANCE: Use a lightweight query to check if data exists (LIMIT 1 instead of COUNT)
has_data = spark.sql("SELECT 1 FROM audit_events LIMIT 1").count() > 0

log(f"\n✅ Temp view 'audit_events' created successfully")
log(f"   All timestamps converted to {TIMEZONE}")

if not has_data:
    log(f"\n⚠️  No modification events found for {filter_description}")
    log(f"   Date range: {START_DATE} to {END_DATE}")
    log(f"   This means no notebooks were created, modified, or deleted in this folder during this period.")
    log(f"   Note: Read-only events like 'openNotebook' are excluded from this audit.")

In [0]:
# Summary of modification events by action type
# PERFORMANCE: This aggregation is efficient as it groups by a low-cardinality column
summary_df = spark.sql("""
SELECT 
  action_name,
  COUNT(*) as event_count,
  COUNT(DISTINCT user_email) as unique_users,
  COUNT(DISTINCT notebook_path) as unique_notebooks,
  MIN(event_date) as first_event,
  MAX(event_date) as last_event
FROM audit_events
GROUP BY action_name
ORDER BY event_count DESC
""")

# Display only in interactive mode
if not IS_JOB_MODE:
    display(summary_df)

In [0]:
# Detailed per-event log showing notebook path, action type, user, and timestamp
# PERFORMANCE: LIMIT prevents memory issues with large result sets
# Set MAX_DETAILED_EVENTS to 9999 to retrieve ALL events without limit

# Build query with conditional LIMIT clause
if MAX_DETAILED_EVENTS == 9999:
    limit_clause = ""  # No limit - retrieve all events
else:
    limit_clause = f"LIMIT {MAX_DETAILED_EVENTS}"

detailed_log_df = spark.sql(f"""
SELECT 
  event_time,
  notebook_path,
  action_name,
  user_email,
  status_code as status,
  request_id
FROM audit_events
ORDER BY event_time DESC
{limit_clause}
""")

# Display only in interactive mode
if not IS_JOB_MODE:
    display(detailed_log_df)

In [0]:
# Summary of modifications by user
# PERFORMANCE: COLLECT_SET can be expensive with many distinct values
# Consider the cardinality of action_name per user (typically low)
user_activity_df = spark.sql("""
SELECT 
  user_email,
  COUNT(*) as total_modifications,
  COUNT(DISTINCT notebook_path) as notebooks_modified,
  MIN(event_date) as first_activity,
  MAX(event_date) as last_activity,
  COLLECT_SET(action_name) as action_types
FROM audit_events
GROUP BY user_email
ORDER BY total_modifications DESC
""")

# Display only in interactive mode
if not IS_JOB_MODE:
    display(user_activity_df)

In [0]:
# Per-notebook modification history
# PERFORMANCE: COLLECT_SET operations can be expensive with high cardinality
# For notebooks with many modifiers, consider limiting or using alternative aggregations
notebook_history_df = spark.sql("""
SELECT 
  notebook_path,
  COUNT(*) as modification_count,
  COUNT(DISTINCT user_email) as unique_modifiers,
  COLLECT_SET(user_email) as modifiers,
  MIN(event_time) as first_modified,
  MAX(event_time) as last_modified,
  COLLECT_SET(action_name) as action_types
FROM audit_events
GROUP BY notebook_path
ORDER BY modification_count DESC
""")

# Display only in interactive mode
if not IS_JOB_MODE:
    display(notebook_history_df)

In [0]:
# Enhanced change tracking: Analyze modification patterns over time
# This provides insights into when notebooks are most actively modified

from pyspark.sql.functions import date_format, hour, dayofweek, count, col

# Modification patterns by day of week and hour
time_pattern_df = spark.sql("""
SELECT 
  DAYOFWEEK(event_time) as day_of_week,
  CASE DAYOFWEEK(event_time)
    WHEN 1 THEN 'Sunday'
    WHEN 2 THEN 'Monday'
    WHEN 3 THEN 'Tuesday'
    WHEN 4 THEN 'Wednesday'
    WHEN 5 THEN 'Thursday'
    WHEN 6 THEN 'Friday'
    WHEN 7 THEN 'Saturday'
  END as day_name,
  HOUR(event_time) as hour_of_day,
  COUNT(*) as modification_count,
  COUNT(DISTINCT notebook_path) as unique_notebooks,
  COUNT(DISTINCT user_email) as unique_users
FROM audit_events
GROUP BY DAYOFWEEK(event_time), HOUR(event_time)
ORDER BY day_of_week, hour_of_day
""")

log("\n📊 Modification Patterns by Time")
log("   Analyzing when notebooks are most actively modified...\n")

if not IS_JOB_MODE:
    display(time_pattern_df)

# Daily modification trends
daily_trend_df = spark.sql("""
SELECT 
  DATE(event_time) as modification_date,
  COUNT(*) as total_modifications,
  COUNT(DISTINCT notebook_path) as notebooks_modified,
  COUNT(DISTINCT user_email) as active_users,
  COLLECT_SET(action_name) as action_types
FROM audit_events
GROUP BY DATE(event_time)
ORDER BY modification_date DESC
""")

log("\n📈 Daily Modification Trends")
log("   Day-by-day breakdown of modification activity...\n")

if not IS_JOB_MODE:
    display(daily_trend_df)

In [0]:
# Stale Notebook Detection: Identify notebooks not modified recently
# This helps with workspace cleanup and compliance

from pyspark.sql import functions as F

log("\n" + "="*60)
log("STALE NOTEBOOK DETECTION")
log("="*60)

if len(notebooks) > 0:
    from pyspark.sql.functions import datediff, current_date, lit, to_date
    
    # Convert notebooks to DataFrame with explicit string conversion for timestamps
    notebooks_simple = [{
        'path': nb['path'],
        'name': nb['name'],
        'language': str(nb.get('language', 'UNKNOWN')),
        'size': int(nb.get('size', 0)) if nb.get('size') else 0,
        'modified_at': str(nb.get('modified_at', '')) if nb.get('modified_at') else None,
        'created_at': str(nb.get('created_at', '')) if nb.get('created_at') else None
    } for nb in notebooks]
    
    notebooks_df = spark.createDataFrame(notebooks_simple)
    
    # Get last modification from audit logs
    last_modified_df = spark.sql("""
    SELECT 
      notebook_path,
      MAX(event_date) as last_modified_date,
      MAX(event_time) as last_modified_time,
      COUNT(*) as total_modifications
    FROM audit_events
    GROUP BY notebook_path
    """)
    
    # Join with notebooks to find stale ones
    notebooks_with_activity = notebooks_df.join(
        last_modified_df,
        notebooks_df.path == last_modified_df.notebook_path,
        'left'
    )
    
    # Calculate days since last modification
    stale_analysis = notebooks_with_activity.withColumn(
        'days_since_modification',
        datediff(current_date(), F.col('last_modified_date'))
    )
    
    # Define stale thresholds
    stale_90_days = stale_analysis.filter(
        (F.col('days_since_modification') >= 90) | F.col('last_modified_date').isNull()
    )
    
    stale_180_days = stale_analysis.filter(
        (F.col('days_since_modification') >= 180) | F.col('last_modified_date').isNull()
    )
    
    stale_365_days = stale_analysis.filter(
        (F.col('days_since_modification') >= 365) | F.col('last_modified_date').isNull()
    )
    
    # No modifications in audit period
    no_activity = stale_analysis.filter(F.col('last_modified_date').isNull())
    
    log(f"\n📊 Stale Notebook Summary:")
    log(f"  Total notebooks scanned: {notebooks_df.count()}")
    log(f"  Not modified in 90+ days: {stale_90_days.count()}")
    log(f"  Not modified in 180+ days: {stale_180_days.count()}")
    log(f"  Not modified in 365+ days: {stale_365_days.count()}")
    log(f"  No activity in audit period: {no_activity.count()}")
    
    if stale_90_days.count() > 0:
        log(f"\n⚠️  COMPLIANCE: {stale_90_days.count()} notebooks may be candidates for cleanup")
        
        if not IS_JOB_MODE:
            log("\nTop 20 stale notebooks (90+ days):")
            display(stale_90_days.select(
                'path', 'language', 'days_since_modification', 'last_modified_date'
            ).orderBy(F.desc('days_since_modification')).limit(20))
        
        log(f"\n💡 RECOMMENDATION: Review stale notebooks for archival or deletion")
        log(f"   Consider workspace cleanup policies for notebooks inactive >90 days")
    else:
        log("\n✓ All notebooks have been modified within 90 days")
    
    # Store for export
    stale_notebooks_df = stale_90_days
    
else:
    log("\n⚠️  No notebooks found - skipping stale detection")
    stale_notebooks_df = spark.createDataFrame([], 'path STRING, language STRING, days_since_modification INT, last_modified_date DATE')

log("="*60)

In [0]:
# Notebook Ownership: Identify who created each notebook
# Helps with accountability and cleanup decisions

log("\n" + "="*60)
log("NOTEBOOK OWNERSHIP IDENTIFICATION")
log("="*60)

if len(notebooks) > 0:
    # Find the first createNotebook event for each notebook
    ownership_df = spark.sql("""
    SELECT 
      notebook_path,
      user_email as owner,
      MIN(event_time) as created_time,
      MIN(event_date) as created_date
    FROM audit_events
    WHERE action_name = 'createNotebook'
    GROUP BY notebook_path, user_email
    """)
    
    # Get the earliest creation event per notebook (in case multiple users created it)
    first_creation = spark.sql("""
    WITH ranked_creations AS (
      SELECT 
        notebook_path,
        user_email as owner,
        event_time as created_time,
        event_date as created_date,
        ROW_NUMBER() OVER (PARTITION BY notebook_path ORDER BY event_time ASC) as rn
      FROM audit_events
      WHERE action_name = 'createNotebook'
    )
    SELECT notebook_path, owner, created_time, created_date
    FROM ranked_creations
    WHERE rn = 1
    """)
    
    ownership_count = first_creation.count()
    
    log(f"\n📋 Ownership Summary:")
    log(f"  Notebooks with identified owners: {ownership_count}")
    log(f"  Notebooks without owner info: {len(notebooks) - ownership_count}")
    
    if ownership_count > 0:
        # Owner statistics
        owner_stats = first_creation.groupBy('owner') \
            .agg(
                F.count('*').alias('notebooks_owned')
            ) \
            .orderBy(F.desc('notebooks_owned'))
        
        log(f"\n👥 Top notebook owners:")
        if not IS_JOB_MODE:
            display(owner_stats.limit(20))
        
        # Notebooks without owners (created before audit period)
        notebooks_df = spark.createDataFrame(notebooks)
        notebooks_without_owners = notebooks_df.join(
            first_creation,
            notebooks_df.path == first_creation.notebook_path,
            'left_anti'
        )
        
        orphaned_count = notebooks_without_owners.count()
        
        if orphaned_count > 0:
            log(f"\n⚠️  {orphaned_count} notebooks have no owner information")
            log(f"   These were likely created before {START_DATE}")
            log(f"   Consider extending the audit date range to capture creation events")
        
        # Store for export
        notebook_ownership_df = first_creation
    else:
        log("\n⚠️  No ownership information found in audit period")
        log(f"   Notebooks may have been created before {START_DATE}")
        notebook_ownership_df = spark.createDataFrame([], 'notebook_path STRING, owner STRING, created_time TIMESTAMP, created_date DATE')
else:
    log("\n⚠️  No notebooks found - skipping ownership identification")
    notebook_ownership_df = spark.createDataFrame([], 'notebook_path STRING, owner STRING, created_time TIMESTAMP, created_date DATE')

log("="*60)

In [0]:
# Execution Failure Analysis: Identify notebooks with failed executions
# Helps identify quality and reliability issues

log("\n" + "="*60)
log("EXECUTION FAILURE ANALYSIS")
log("="*60)

# Query for failed runCommand events
failed_executions_df = spark.sql("""
SELECT 
  notebook_path,
  user_email,
  event_time,
  event_date,
  status_code,
  request_id
FROM audit_events
WHERE action_name = 'runCommand' 
  AND (status_code != '200' OR status_code IS NULL)
ORDER BY event_time DESC
""")

failed_count = failed_executions_df.count()

log(f"\n🔍 Execution Failure Summary:")
log(f"  Total failed executions: {failed_count}")

if failed_count > 0:
    log(f"\n⚠️  QUALITY ALERT: Notebooks with execution failures detected")
    
    # Notebooks with most failures
    failure_summary = failed_executions_df.groupBy('notebook_path') \
        .agg(
            F.count('*').alias('failure_count'),
            F.countDistinct('user_email').alias('users_affected'),
            F.max('event_date').alias('last_failure_date')
        ) \
        .orderBy(F.desc('failure_count'))
    
    log(f"  Notebooks with failures: {failure_summary.count()}")
    
    if not IS_JOB_MODE:
        log("\nTop notebooks by failure count:")
        display(failure_summary.limit(20))
    
    # Users with most failures
    user_failures = failed_executions_df.groupBy('user_email') \
        .agg(
            F.count('*').alias('failure_count'),
            F.countDistinct('notebook_path').alias('notebooks_affected')
        ) \
        .orderBy(F.desc('failure_count'))
    
    if not IS_JOB_MODE:
        log("\nUsers with most execution failures:")
        display(user_failures.limit(10))
    
    log(f"\n💡 RECOMMENDATION: Investigate notebooks with high failure rates")
    log(f"   Failed executions may indicate code quality or data issues")
    
    # Store for export
    notebook_failures_df = failure_summary
else:
    log("\n✓ No execution failures detected in audit period")
    log(f"   All runCommand events completed successfully")
    notebook_failures_df = spark.createDataFrame([], 'notebook_path STRING, failure_count LONG, users_affected LONG, last_failure_date DATE')

log("="*60)

In [0]:
# Language Distribution and Folder Statistics
# Provides insights into notebook composition and organization

from pyspark.sql import functions as F

log("\n" + "="*60)
log("LANGUAGE DISTRIBUTION & FOLDER STATISTICS")
log("="*60)

if len(notebooks) > 0:
    # Reuse notebooks_df from previous cell if available, otherwise create it
    if 'notebooks_df' not in dir():
        notebooks_simple = [{
            'path': nb['path'],
            'name': nb['name'],
            'language': str(nb.get('language', 'UNKNOWN')),
            'size': int(nb.get('size', 0)) if nb.get('size') else 0
        } for nb in notebooks]
        notebooks_df = spark.createDataFrame(notebooks_simple)
    
    # Language distribution
    log("\n📊 Language Distribution:")
    language_dist = notebooks_df.groupBy('language') \
        .agg(F.count('*').alias('notebook_count')) \
        .orderBy(F.desc('notebook_count'))
    
    for row in language_dist.collect():
        percentage = (row['notebook_count'] / len(notebooks)) * 100
        log(f"  {row['language']}: {row['notebook_count']} notebooks ({percentage:.1f}%)")
    
    if not IS_JOB_MODE:
        display(language_dist)
    
    # Folder-level statistics
    log("\n📁 Folder-Level Statistics:")
    
    # Extract folder path from notebook path
    notebooks_with_folder = notebooks_df.withColumn(
        'folder_path',
        F.expr("substring(path, 1, length(path) - length(name) - 1)")
    )
    
    folder_stats = notebooks_with_folder.groupBy('folder_path') \
        .agg(
            F.count('*').alias('notebook_count'),
            F.collect_set('language').alias('languages_used')
        ) \
        .orderBy(F.desc('notebook_count'))
    
    log(f"  Total folders: {folder_stats.count()}")
    log(f"  Average notebooks per folder: {len(notebooks) / folder_stats.count():.1f}")
    
    if not IS_JOB_MODE:
        log("\nTop folders by notebook count:")
        display(folder_stats.limit(20))
    
    # Notebook size analysis (if available)
    if 'size' in notebooks_df.columns:
        size_stats = notebooks_df.filter(F.col('size') > 0).agg(
            F.avg('size').alias('avg_size'),
            F.max('size').alias('max_size'),
            F.min('size').alias('min_size'),
            F.sum('size').alias('total_size')
        ).collect()
        
        if len(size_stats) > 0 and size_stats[0].avg_size is not None:
            size_row = size_stats[0]
            
            log(f"\n💾 Notebook Size Analysis:")
            log(f"  Average size: {size_row.avg_size / 1024:.1f} KB")
            log(f"  Largest notebook: {size_row.max_size / 1024:.1f} KB")
            log(f"  Total size: {size_row.total_size / (1024 * 1024):.1f} MB")
            
            # Identify large notebooks (>1MB)
            large_notebooks = notebooks_df.filter(F.col('size') > 1024 * 1024)
            
            if large_notebooks.count() > 0:
                log(f"\n⚠️  Large notebooks (>1MB): {large_notebooks.count()}")
                if not IS_JOB_MODE:
                    display(large_notebooks.select('path', 'language', (F.col('size') / 1024).alias('size_kb')).orderBy(F.desc('size')))
    
    # Store for export
    language_distribution_df = language_dist
    folder_statistics_df = folder_stats
    
else:
    log("\n⚠️  No notebooks found - skipping analysis")
    language_distribution_df = spark.createDataFrame([], 'language STRING, notebook_count LONG')
    folder_statistics_df = spark.createDataFrame([], 'folder_path STRING, notebook_count LONG, languages_used ARRAY<STRING>')

log("="*60)

In [0]:
# Security and Compliance Analysis
# Detect after-hours modifications, deletions, and external users

log("\n" + "="*60)
log("SECURITY & COMPLIANCE ANALYSIS")
log("="*60)

# 1. After-hours modifications (outside 8 AM - 6 PM)
log("\n1. After-Hours Modifications:")

after_hours_df = spark.sql("""
SELECT 
  notebook_path,
  user_email,
  event_time,
  action_name,
  HOUR(event_time) as hour_of_day
FROM audit_events
WHERE HOUR(event_time) < 8 OR HOUR(event_time) >= 18
ORDER BY event_time DESC
""")

after_hours_count = after_hours_df.count()

log(f"  After-hours modifications: {after_hours_count}")

if after_hours_count > 0:
    log(f"\n⚠️  SECURITY REVIEW: Modifications outside business hours (8 AM - 6 PM)")
    
    # Users with most after-hours activity
    after_hours_users = after_hours_df.groupBy('user_email') \
        .agg(F.count('*').alias('after_hours_count')) \
        .orderBy(F.desc('after_hours_count'))
    
    if not IS_JOB_MODE:
        log("\nUsers with after-hours activity:")
        display(after_hours_users.limit(10))
else:
    log("  ✓ All modifications occurred during business hours")

# 2. Deletion events (high-risk)
log("\n2. Notebook Deletions:")

deletions_df = spark.sql("""
SELECT 
  notebook_path,
  user_email,
  event_time,
  event_date
FROM audit_events
WHERE action_name = 'deleteNotebook'
ORDER BY event_time DESC
""")

deletion_count = deletions_df.count()

log(f"  Notebooks deleted: {deletion_count}")

if deletion_count > 0:
    log(f"\n⚠️  HIGH-RISK: {deletion_count} notebooks were deleted in audit period")
    
    if not IS_JOB_MODE:
        log("\nDeleted notebooks:")
        display(deletions_df)
    
    # Users who deleted notebooks
    deleters = deletions_df.groupBy('user_email') \
        .agg(F.count('*').alias('deletions')) \
        .orderBy(F.desc('deletions'))
    
    log(f"\n  Users who deleted notebooks: {deleters.count()}")
    
    log(f"\n💡 RECOMMENDATION: Review deletion events for compliance")
    log(f"   Ensure deletions were authorized and documented")
else:
    log("  ✓ No notebooks were deleted in audit period")

# 3. External user modifications
log("\n3. External User Activity:")

COMPANY_DOMAIN = 'bat.com'  # Customize for your organization

external_users_df = spark.sql(f"""
SELECT 
  user_email,
  COUNT(*) as modification_count,
  COUNT(DISTINCT notebook_path) as notebooks_modified,
  MIN(event_date) as first_activity,
  MAX(event_date) as last_activity
FROM audit_events
WHERE user_email NOT LIKE '%@{COMPANY_DOMAIN}'
GROUP BY user_email
ORDER BY modification_count DESC
""")

external_count = external_users_df.count()

log(f"  External users (non-@{COMPANY_DOMAIN}): {external_count}")

if external_count > 0:
    log(f"\n⚠️  SECURITY REVIEW: External users modified notebooks")
    
    if not IS_JOB_MODE:
        display(external_users_df)
    
    log(f"\n💡 RECOMMENDATION: Review external user access")
    log(f"   Verify external users have appropriate business justification")
else:
    log(f"  ✓ All modifications by @{COMPANY_DOMAIN} users")

# Store for export
after_hours_modifications_df = after_hours_df
notebook_deletions_df = deletions_df
external_user_activity_df = external_users_df

log("="*60)

In [0]:
# Collaboration Metrics: Analyze team collaboration patterns
# Identifies notebooks with multiple editors and collaboration intensity

log("\n" + "="*60)
log("COLLABORATION METRICS")
log("="*60)

# Notebooks with multiple editors
collaboration_df = spark.sql("""
SELECT 
  notebook_path,
  COUNT(DISTINCT user_email) as unique_editors,
  COUNT(*) as total_modifications,
  COLLECT_SET(user_email) as editors,
  MIN(event_date) as first_modified,
  MAX(event_date) as last_modified
FROM audit_events
GROUP BY notebook_path
HAVING COUNT(DISTINCT user_email) > 1
ORDER BY unique_editors DESC, total_modifications DESC
""")

collab_count = collaboration_df.count()

log(f"\n🤝 Collaboration Summary:")
log(f"  Notebooks with multiple editors: {collab_count}")

if collab_count > 0:
    # Highly collaborative notebooks (3+ editors)
    highly_collaborative = collaboration_df.filter(F.col('unique_editors') >= 3)
    
    log(f"  Highly collaborative (3+ editors): {highly_collaborative.count()}")
    
    if not IS_JOB_MODE:
        log("\nMost collaborative notebooks:")
        display(collaboration_df.limit(20))
    
    # Collaboration statistics
    collab_stats = collaboration_df.agg(
        F.avg('unique_editors').alias('avg_editors'),
        F.max('unique_editors').alias('max_editors'),
        F.avg('total_modifications').alias('avg_modifications')
    ).collect()[0]
    
    log(f"\n📊 Collaboration Statistics:")
    log(f"  Average editors per collaborative notebook: {collab_stats.avg_editors:.1f}")
    log(f"  Most editors on single notebook: {collab_stats.max_editors}")
    log(f"  Average modifications per collaborative notebook: {collab_stats.avg_modifications:.1f}")
    
    log(f"\n💡 INSIGHT: High collaboration indicates active team development")
    log(f"   Consider code review processes for highly collaborative notebooks")
else:
    log("\n  ℹ️  No collaborative notebooks found (all notebooks have single editor)")

# Store for export
collaborative_notebooks_df = collaboration_df

log("="*60)

In [0]:
# Visualization: Activity heatmaps and trend charts
# Visual representation of modification patterns

if not IS_JOB_MODE:
    import matplotlib.pyplot as plt
    import numpy as np
    
    log("\n" + "="*60)
    log("ACTIVITY VISUALIZATIONS")
    log("="*60)
    
    # Get time pattern data
    time_data = time_pattern_df.toPandas()
    
    if len(time_data) > 0:
        # Create figure with subplots
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        
        # Chart 1: Activity by day of week
        day_summary = time_data.groupby('day_name')['modification_count'].sum().reindex(
            ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
        )
        axes[0, 0].bar(day_summary.index, day_summary.values, color='steelblue')
        axes[0, 0].set_title('Modifications by Day of Week', fontsize=14, fontweight='bold')
        axes[0, 0].set_xlabel('Day')
        axes[0, 0].set_ylabel('Modification Count')
        axes[0, 0].tick_params(axis='x', rotation=45)
        axes[0, 0].grid(axis='y', alpha=0.3)
        
        # Chart 2: Activity by hour of day
        hour_summary = time_data.groupby('hour_of_day')['modification_count'].sum().sort_index()
        axes[0, 1].plot(hour_summary.index, hour_summary.values, marker='o', linewidth=2, color='darkgreen')
        axes[0, 1].set_title('Modifications by Hour of Day', fontsize=14, fontweight='bold')
        axes[0, 1].set_xlabel('Hour (24-hour format)')
        axes[0, 1].set_ylabel('Modification Count')
        axes[0, 1].grid(alpha=0.3)
        axes[0, 1].set_xticks(range(0, 24, 2))
        
        # Chart 3: Heatmap (day vs hour)
        pivot_data = time_data.pivot_table(
            values='modification_count',
            index='day_name',
            columns='hour_of_day',
            aggfunc='sum',
            fill_value=0
        ).reindex(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
        
        im = axes[1, 0].imshow(pivot_data.values, cmap='YlOrRd', aspect='auto')
        axes[1, 0].set_title('Activity Heatmap (Day vs Hour)', fontsize=14, fontweight='bold')
        axes[1, 0].set_xlabel('Hour of Day')
        axes[1, 0].set_ylabel('Day of Week')
        axes[1, 0].set_yticks(range(len(pivot_data.index)))
        axes[1, 0].set_yticklabels(pivot_data.index)
        axes[1, 0].set_xticks(range(0, 24, 2))
        axes[1, 0].set_xticklabels(range(0, 24, 2))
        plt.colorbar(im, ax=axes[1, 0], label='Modifications')
        
        # Chart 4: Daily trend
        daily_data = daily_trend_df.toPandas()
        if len(daily_data) > 0:
            axes[1, 1].plot(daily_data['event_date'], daily_data['modification_count'], 
                          marker='o', linewidth=2, color='purple')
            axes[1, 1].set_title('Daily Modification Trend', fontsize=14, fontweight='bold')
            axes[1, 1].set_xlabel('Date')
            axes[1, 1].set_ylabel('Modification Count')
            axes[1, 1].grid(alpha=0.3)
            axes[1, 1].tick_params(axis='x', rotation=45)
        
        plt.tight_layout()
        plt.show()
        
        log("\n✓ Activity visualizations generated")
    else:
        log("\n⚠️  No time pattern data available for visualization")
else:
    log("\n⏭️  Visualizations skipped in job mode")

In [0]:
# Enhanced Excel Export with Summary Dashboard
# Comprehensive workbook with multiple analysis sheets

ENABLE_EXCEL_EXPORT = False  # Set to True to enable Excel export

if ENABLE_EXCEL_EXPORT:
    log("\n" + "="*60)
    log("ENHANCED EXCEL EXPORT")
    log("="*60)
    
    try:
        import tempfile
        
        # Use temp directory for serverless compatibility
        temp_dir = tempfile.mkdtemp()
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        temp_excel_file = f"{temp_dir}/folder_audit_{timestamp}.xlsx"
        
        log(f"\nCreating Excel workbook with multiple sheets...")
        
        with pd.ExcelWriter(temp_excel_file, engine='openpyxl') as writer:
            
            # Sheet 1: Executive Summary
            exec_summary = pd.DataFrame([{
                'Metric': 'Audit Period',
                'Value': f"{START_DATE} to {END_DATE}"
            }, {
                'Metric': 'Folder Scanned',
                'Value': BASE_PATH if BASE_PATH else 'All Workspace Folders'
            }, {
                'Metric': 'Total Notebooks',
                'Value': len(notebooks)
            }, {
                'Metric': 'Total Modifications',
                'Value': spark.sql("SELECT COUNT(*) as cnt FROM audit_events").collect()[0]['cnt']
            }, {
                'Metric': 'Unique Users',
                'Value': spark.sql("SELECT COUNT(DISTINCT user_email) as cnt FROM audit_events").collect()[0]['cnt']
            }, {
                'Metric': 'Stale Notebooks (90+ days)',
                'Value': stale_notebooks_df.count() if 'stale_notebooks_df' in dir() else 'N/A'
            }, {
                'Metric': 'Execution Failures',
                'Value': notebook_failures_df.count() if 'notebook_failures_df' in dir() else 'N/A'
            }, {
                'Metric': 'After-Hours Modifications',
                'Value': after_hours_count
            }, {
                'Metric': 'Deletions',
                'Value': deletions_df.count() if 'deletions_df' in dir() else 'N/A'
            }, {
                'Metric': 'Audit Timestamp',
                'Value': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }])
            
            exec_summary.to_excel(writer, sheet_name='Executive Summary', index=False)
            log("  ✓ Sheet 1: Executive Summary")
            
            # Sheet 2: Audit Events Summary
            summary_df.toPandas().to_excel(writer, sheet_name='Events Summary', index=False)
            log("  ✓ Sheet 2: Events Summary")
            
            # Sheet 3: User Activity
            user_activity_df.toPandas().to_excel(writer, sheet_name='User Activity', index=False)
            log("  ✓ Sheet 3: User Activity")
            
            # Sheet 4: Notebook History
            notebook_history_df.toPandas().to_excel(writer, sheet_name='Notebook History', index=False)
            log("  ✓ Sheet 4: Notebook History")
            
            # Sheet 5: Stale Notebooks
            if 'stale_notebooks_df' in dir():
                stale_notebooks_df.toPandas().to_excel(writer, sheet_name='Stale Notebooks', index=False)
                log("  ✓ Sheet 5: Stale Notebooks")
            
            # Sheet 6: Ownership
            if 'notebook_ownership_df' in dir():
                notebook_ownership_df.toPandas().to_excel(writer, sheet_name='Ownership', index=False)
                log("  ✓ Sheet 6: Ownership")
            
            # Sheet 7: Execution Failures
            if 'notebook_failures_df' in dir():
                notebook_failures_df.toPandas().to_excel(writer, sheet_name='Execution Failures', index=False)
                log("  ✓ Sheet 7: Execution Failures")
            
            # Sheet 8: Language Distribution
            if 'language_distribution_df' in dir():
                language_distribution_df.toPandas().to_excel(writer, sheet_name='Language Distribution', index=False)
                log("  ✓ Sheet 8: Language Distribution")
            
            # Sheet 9: Security Alerts
            security_summary = pd.DataFrame([{
                'Alert Type': 'After-Hours Modifications',
                'Count': after_hours_count,
                'Severity': 'Medium' if after_hours_count > 0 else 'None'
            }, {
                'Alert Type': 'Notebook Deletions',
                'Count': deletion_count,
                'Severity': 'High' if deletion_count > 0 else 'None'
            }, {
                'Alert Type': 'External User Activity',
                'Count': external_count,
                'Severity': 'Medium' if external_count > 0 else 'None'
            }])
            security_summary.to_excel(writer, sheet_name='Security Alerts', index=False)
            log("  ✓ Sheet 9: Security Alerts")
        
        file_size_kb = os.path.getsize(temp_excel_file) / 1024
        log(f"\n✓ Enhanced Excel export complete!")
        log(f"  File: {temp_excel_file}")
        log(f"  Sheets: 9")
        log(f"  Size: {file_size_kb:.1f} KB")
        
    except Exception as e:
        log(f"❌ Excel export failed: {str(e)}")
else:
    log("\nℹ️  Excel export disabled (ENABLE_EXCEL_EXPORT=False)")
    log("   Set ENABLE_EXCEL_EXPORT=True in this cell to enable")

In [0]:
# JSON Export: Structured data for API integration and programmatic access

ENABLE_JSON_EXPORT = False  # Set to True to enable JSON export

if ENABLE_JSON_EXPORT:
    log("\n" + "="*60)
    log("JSON EXPORT")
    log("="*60)
    
    try:
        import json
        import tempfile
        
        # Use temp directory for serverless compatibility
        temp_dir = tempfile.mkdtemp()
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        json_file = f"{temp_dir}/folder_audit_{timestamp}.json"
        
        # Prepare JSON structure
        export_data = {
            'metadata': {
                'audit_timestamp': datetime.now().isoformat(),
                'audit_period_start': START_DATE,
                'audit_period_end': END_DATE,
                'folder_scanned': BASE_PATH if BASE_PATH else 'All Workspace Folders',
                'timezone': TIMEZONE,
                'execution_mode': 'Job' if IS_JOB_MODE else 'Interactive',
                'compute_type': 'Serverless' if IS_SERVERLESS else 'Traditional'
            },
            'summary': {
                'total_notebooks': len(notebooks),
                'total_modifications': spark.sql("SELECT COUNT(*) as cnt FROM audit_events").collect()[0]['cnt'],
                'unique_users': spark.sql("SELECT COUNT(DISTINCT user_email) as cnt FROM audit_events").collect()[0]['cnt'],
                'stale_notebooks': stale_notebooks_df.count() if 'stale_notebooks_df' in dir() else 0,
                'execution_failures': notebook_failures_df.count() if 'notebook_failures_df' in dir() else 0,
                'after_hours_modifications': after_hours_count if 'after_hours_count' in dir() else 0,
                'deletions': deletion_count if 'deletion_count' in dir() else 0,
                'collaborative_notebooks': collab_count if 'collab_count' in dir() else 0
            },
            'notebooks': notebooks,
            'audit_events_summary': summary_df.toPandas().to_dict('records'),
            'user_activity': user_activity_df.toPandas().to_dict('records'),
            'language_distribution': language_distribution_df.toPandas().to_dict('records') if 'language_distribution_df' in dir() else []
        }
        
        # Write JSON file
        with open(json_file, 'w') as f:
            json.dump(export_data, f, indent=2, default=str)
        
        file_size_kb = os.path.getsize(json_file) / 1024
        log(f"\n✓ JSON export complete!")
        log(f"  File: {json_file}")
        log(f"  Size: {file_size_kb:.1f} KB")
        
    except Exception as e:
        log(f"❌ JSON export failed: {str(e)}")
else:
    log("\nℹ️  JSON export disabled (ENABLE_JSON_EXPORT=False)")
    log("   Set ENABLE_JSON_EXPORT=True in this cell to enable")

In [0]:
# Content Snapshot: Combine current notebook metadata with audit history
# This provides a point-in-time view of notebook state with modification history

if ENABLE_CONTENT_SNAPSHOT and len(notebooks) > 0:
    from pyspark.sql.functions import lit, current_timestamp
    
    log("\n📸 Creating Content Snapshot")
    log("   Combining current notebook metadata with audit history...\n")
    
    # Convert notebooks list to DataFrame
    notebooks_df = spark.createDataFrame(notebooks)
    
    # Create a comprehensive view joining notebook metadata with audit summary
    snapshot_df = spark.sql("""
    SELECT 
      notebook_path,
      COUNT(*) as total_modifications,
      COUNT(DISTINCT user_email) as unique_modifiers,
      MIN(event_time) as first_modification,
      MAX(event_time) as last_modification,
      DATEDIFF(MAX(event_time), MIN(event_time)) as days_active,
      COLLECT_SET(action_name) as modification_types,
      COLLECT_SET(user_email) as modifier_list
    FROM audit_events
    GROUP BY notebook_path
    """)
    
    # Join with current notebook metadata
    if 'modified_at' in notebooks_df.columns:
        enriched_snapshot_df = notebooks_df.join(
            snapshot_df,
            notebooks_df.path == snapshot_df.notebook_path,
            'left'
        ).select(
            notebooks_df.path.alias('notebook_path'),
            notebooks_df.name.alias('notebook_name'),
            notebooks_df.language,
            notebooks_df.modified_at.alias('current_modified_at'),
            notebooks_df.size.alias('current_size_bytes'),
            snapshot_df.total_modifications,
            snapshot_df.unique_modifiers,
            snapshot_df.first_modification,
            snapshot_df.last_modification,
            snapshot_df.days_active,
            snapshot_df.modification_types,
            snapshot_df.modifier_list
        ).orderBy(col('total_modifications').desc_nulls_last())
    else:
        enriched_snapshot_df = notebooks_df.join(
            snapshot_df,
            notebooks_df.path == snapshot_df.notebook_path,
            'left'
        ).select(
            notebooks_df.path.alias('notebook_path'),
            notebooks_df.name.alias('notebook_name'),
            notebooks_df.language,
            snapshot_df.total_modifications,
            snapshot_df.unique_modifiers,
            snapshot_df.first_modification,
            snapshot_df.last_modification,
            snapshot_df.days_active,
            snapshot_df.modification_types,
            snapshot_df.modifier_list
        ).orderBy(col('total_modifications').desc_nulls_last())
    
    log("✅ Content snapshot created successfully")
    log(f"   Captured metadata for {notebooks_df.count()} notebooks")
    log(f"   Matched with audit history for {snapshot_df.count()} notebooks\n")
    
    if not IS_JOB_MODE:
        display(enriched_snapshot_df)
    
    # Store for potential export
    content_snapshot_df = enriched_snapshot_df
else:
    log("\n⏭️  Content snapshot disabled or no notebooks found")
    log("   Set ENABLE_CONTENT_SNAPSHOT = True to enable this feature\n")
    content_snapshot_df = None

In [0]:
# Export audit data to Delta table for long-term retention and historical analysis
# This enables tracking changes over multiple audit runs

if ENABLE_DELTA_EXPORT:
    from pyspark.sql.functions import current_timestamp, lit
    
    log("\n💾 Exporting Audit Data to Delta Table")
    log(f"   Target table: {DELTA_TABLE_NAME}")
    log("="*60)
    
    try:
        # Prepare the export DataFrame with metadata
        export_df = spark.sql(f"""
        SELECT 
          event_time,
          event_date,
          notebook_path,
          action_name,
          user_email,
          status_code,
          request_id,
          notebook_id
        FROM audit_events
        ORDER BY event_time DESC
        """)
        
        # Add audit run metadata
        export_df = export_df \
            .withColumn('audit_run_timestamp', current_timestamp()) \
            .withColumn('audit_start_date', lit(START_DATE)) \
            .withColumn('audit_end_date', lit(END_DATE)) \
            .withColumn('audit_base_path', lit(BASE_PATH if BASE_PATH else 'ALL_FOLDERS'))
        
        # Write to Delta table with merge capability
        # Using append mode to accumulate historical data
        export_df.write \
            .format('delta') \
            .mode('append') \
            .option('mergeSchema', 'true') \
            .saveAsTable(DELTA_TABLE_NAME)
        
        record_count = export_df.count()
        
        log(f"\n✅ Successfully exported {record_count} audit records")
        log(f"   Table: {DELTA_TABLE_NAME}")
        log(f"   Mode: APPEND (historical accumulation)")
        log(f"   Schema merge: ENABLED")
        
        # Also export content snapshot if available
        if ENABLE_CONTENT_SNAPSHOT and content_snapshot_df is not None:
            snapshot_table_name = DELTA_TABLE_NAME.replace('_history', '_snapshot')
            
            content_snapshot_df \
                .withColumn('snapshot_timestamp', current_timestamp()) \
                .withColumn('audit_base_path', lit(BASE_PATH if BASE_PATH else 'ALL_FOLDERS')) \
                .write \
                .format('delta') \
                .mode('overwrite') \
                .option('overwriteSchema', 'true') \
                .saveAsTable(snapshot_table_name)
            
            log(f"\n✅ Successfully exported content snapshot")
            log(f"   Table: {snapshot_table_name}")
            log(f"   Mode: OVERWRITE (latest snapshot)")
        
        log("\n📊 Query your audit history:")
        log(f"   SELECT * FROM {DELTA_TABLE_NAME} WHERE audit_run_timestamp >= current_date()")
        
    except Exception as e:
        log(f"\n❌ Failed to export to Delta table: {e}")
        log(f"   Please verify:")
        log(f"   1. You have CREATE TABLE permissions")
        log(f"   2. The catalog and schema exist: {DELTA_TABLE_NAME.rsplit('.', 1)[0]}")
        log(f"   3. The table name is valid")
        raise
else:
    log("\n⏭️  Delta table export disabled")
    log("   Set ENABLE_DELTA_EXPORT = True to enable long-term retention\n")

In [0]:
# Create a comprehensive audit report combining all data
from pyspark.sql.functions import col

# Determine folder description
if BASE_PATH == "" or BASE_PATH is None:
    folder_description = "All Workspace Folders"
else:
    folder_description = BASE_PATH

# Get current date/time when report was generated in the configured timezone
# PERFORMANCE: Use Spark SQL for timezone conversion instead of Python datetime
report_time_df = spark.sql(f"SELECT from_utc_timestamp(current_timestamp(), '{TIMEZONE}') as report_time")
report_generated = report_time_df.collect()[0]['report_time'].strftime("%Y-%m-%d %H:%M:%S")

log("\n" + "="*80)
log("ACCESS REVIEW AUDIT REPORT")
log("="*80)
log(f"\nReport Generated: {report_generated}")
log(f"Folder: {folder_description}")
log(f"Date Range: {START_DATE} to {END_DATE}")
log(f"Timezone: {TIMEZONE}")

# Display the detailed event log
if MAX_DETAILED_EVENTS == 9999:
    log("\nDetailed Modification Events:")
    log("(Showing ALL events - no limit applied)\n")
else:
    log("\nDetailed Modification Events:")
    log(f"(Showing up to {MAX_DETAILED_EVENTS} most recent events)\n")

# PERFORMANCE: Select only needed columns instead of SELECT *
# Build query with conditional LIMIT clause
if MAX_DETAILED_EVENTS == 9999:
    limit_clause = ""  # No limit - retrieve all events
else:
    limit_clause = f"LIMIT {MAX_DETAILED_EVENTS}"

final_report_df = spark.sql(f"""
  SELECT 
    event_time,
    notebook_path, 
    action_name,
    user_email,
    status_code
  FROM audit_events
  ORDER BY event_time DESC
  {limit_clause}
""")

# Display only in interactive mode
if not IS_JOB_MODE:
    display(final_report_df)
else:
    # In job mode, you might want to write to a Delta table instead
    # Example: final_report_df.write.mode("overwrite").saveAsTable("audit_reports.notebook_modifications")
    pass

In [0]:
# Clean up cached data to free memory
# Run this cell when you're done with the audit report

try:
    spark.sql("UNCACHE TABLE IF EXISTS audit_events")
    log("✅ Cached data cleared successfully")
except Exception as e:
    log(f"⚠️  Note: {e}")

## 🎉 Audit Complete!

### What You Have Now

This notebook has generated comprehensive audit reports with:

* **📄 Detailed Event Logs**: Every modification event with user, timestamp, and action type
* **📊 Summary Statistics**: Aggregated views by action type, user, and notebook
* **⏰ Time-Based Analysis**: Modification patterns by day of week and hour
* **📸 Content Snapshots**: Point-in-time view of notebook state with history
* **💾 Long-Term Retention**: Optional Delta table export for historical analysis

---

### Querying Exported Data

If you enabled `ENABLE_DELTA_EXPORT = True`, you can query your audit history:

```sql
-- View all audit events from today's run
SELECT * FROM main.default.notebook_audit_history 
WHERE audit_run_timestamp >= current_date()
ORDER BY event_time DESC;

-- Find all modifications to a specific notebook
SELECT * FROM main.default.notebook_audit_history 
WHERE notebook_path LIKE '%YourNotebookName%'
ORDER BY event_time DESC;

-- Analyze modification trends over time
SELECT 
  DATE(audit_run_timestamp) as audit_date,
  COUNT(*) as total_events,
  COUNT(DISTINCT notebook_path) as notebooks_modified,
  COUNT(DISTINCT user_email) as active_users
FROM main.default.notebook_audit_history
GROUP BY DATE(audit_run_timestamp)
ORDER BY audit_date DESC;

-- View current snapshot
SELECT * FROM main.default.notebook_audit_snapshot
ORDER BY total_modifications DESC;
```

---

### Scheduling This Notebook

To run this audit automatically:

1. **Create a Job**: Go to Workflows → Create Job
2. **Add this notebook** as a task
3. **Set schedule**: Daily, weekly, or monthly
4. **Enable Delta export**: Set `ENABLE_DELTA_EXPORT = True`
5. **Configure alerts**: Add email notifications for job completion

The notebook will automatically run in job mode (no display output) and export results to Delta tables.

---

### Customization Tips

* **Narrow the scope**: Set `BASE_PATH` to specific folders for faster execution
* **Adjust date range**: Use shorter periods for recent activity analysis
* **Increase detail**: Set `MAX_DETAILED_EVENTS = 9999` for complete history
* **Add filters**: Modify SQL queries to focus on specific users or action types
* **Create dashboards**: Use the Delta tables as data sources for Lakeview dashboards

---

### Compliance & Security

This audit framework helps with:

* ✅ **SOX Compliance**: Track all changes to critical notebooks
* ✅ **Access Reviews**: Identify who modified what and when
* ✅ **Security Monitoring**: Detect unusual modification patterns
* ✅ **Change Management**: Document notebook lifecycle
* ✅ **Team Analytics**: Understand collaboration patterns

---

### Need Help?

Refer to the configuration section in Cell 3 to:
* Change audit scope and date range
* Enable/disable features
* Configure Delta table export
* Adjust performance settings